In [22]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd
import datetime as dt
import functools

In [148]:
# initialize data set. aum = assets under management
columns = ['as_of_date', 'cash_flow', 'aum']
rows = [['30-03-18', 56, 56], 
        ['31-03-18', 10, 66.3], 
        ['01-04-18', 0, 66.2], 
        ['02-04-18', -5, 59]]

# Create the pandas DataFrame
df = pd.DataFrame(rows, columns=columns)
  
# print dataframe.
display(df)

,as_of_date,cash_flow,aum
0,30-03-18,56,56.0
1,31-03-18,10,66.3
2,01-04-18,0,66.2
3,02-04-18,-5,59.0


In [192]:
 # Add the aum of the day before 
df['aum_before_cash_flow'] = df['aum'].shift(1).fillna(0)
# print dataframe.
display(df)

,as_of_date,cash_flow,aum,aum_before_cash_flow
0,01-03-18,5,5,0.0
1,01-04-18,0,15,5.0
2,01-05-18,10,60,15.0
3,01-06-18,200,262,60.0
4,01-07-18,0,206,262.0
5,01-08-18,-30,168,206.0


Note that with this code, we automatically assume that the initial aum is zero. If this is not the case, you would need to make adjustments, like adding a dummy transaction in the beginning or having the inital amount be specified.

In [193]:
v0 = df['aum_before_cash_flow'].iloc[0]
print(f'The value at the beginning of the period v0 is {v0}')

v1 = df['aum'].iloc[-1]
print(f'The value at the end of the period v1 is {v1}')




The value at the beginning of the period v0 is 0.0
The value at the end of the period v1 is 168
Total days is 6


In [194]:
# Prepare dates for calculation
df['as_of_date'] = pd.to_datetime(df.as_of_date, format='%d-%m-%y')
start_date = df['as_of_date'].iloc[0]
df['days_from_begining'] = (df['as_of_date'] - start_date).dt.days
display(df)

,as_of_date,cash_flow,aum,aum_before_cash_flow,days_from_begining
0,2018-03-01,5,5,0.0,0
1,2018-04-01,0,15,5.0,31
2,2018-05-01,10,60,15.0,61
3,2018-06-01,200,262,60.0,92
4,2018-07-01,0,206,262.0,122
5,2018-08-01,-30,168,206.0,153


Daily time weighted return equation:
$$
\Pi(1 + R_i) - 1
$$

With $R_i$ being the basic return between each cash flow, calculated as follows for positive cash flows:

$$
R_i = V_i / (V_{i-1} + C_i) - 1
$$

and for negative cash flows:
$$
R_i = (V_i + C_i) / V_{i-1} - 1
$$

In [186]:
#calculate the return for each section
def calcreturn(aum, cf, prev):
    if(cf == 0):
        return 1 if prev == 0 else aum/prev
    elif(cf > 0):
        return 1 if prev + cf == 0 else (aum / (prev + cf)) #should be >0?
        
    else:
        return ((aum - cf)/ (prev)) #technically seems like it should be +1? But that ruins it all so

df['return_since_last'] = np.vectorize(calcreturn)(df['aum'], df['cash_flow'], df['aum_before_cash_flow'])

In [187]:
# Link the returns
linked = functools.reduce(lambda a, b: a*b, df['return_since_last']) - 1

print(f'The daily weighted time weighted return is {linked}.')

  as_of_date  cash_flow   aum  aum_before_cash_flow  days_from_begining  return_since_last
0 2018-03-30         56  56.0                   0.0                   0           1.000000
1 2018-03-31         10  66.3                  56.0                   1           1.004545
2 2018-04-01          0  66.2                  66.3                   2           0.998492
3 2018-04-02         -5  59.0                  66.2                   3           0.966767
4 2018-07-04        -59   0.0                  59.0                  96           1.000000
5 2018-07-22          0   1.0                   0.0                 114           1.000000
The daily weighted time weighted return is -0.030303030303030498.


Note that this number accurately reflects how the assets in the portfolio performed, not the money itself. Let's take a look at some examples for how this could be confusing, and how to rectify them. Try copying the following table into the "# initialize data set" area above, and rerun all the code. 

In [176]:
# initialize data set. aum = assets under management
columns = ['as_of_date', 'cash_flow', 'aum']
rows = [['30-03-18', 56, 56], 
        ['31-03-18', 10, 66.3], 
        ['01-04-18', 0, 66.2], 
        ['02-04-18', -5, 59],
        ['04-07-18', -59, 0],
        ['22-07-18', 0, 1]]

df = pd.DataFrame(rows, columns=columns)

You should get an error, because in finding the return between 04-07-2018 and 22-07-2018, we have recieved a return of 1 on an investment of $0, which is an infinite return. This is often caused by recieving dividends late, after the assets have all been sold. Ideally, these dividends would be changed in the dataset to reflect when they were actually earned instead of when they were received; however, in this case, we will simply choose to remove them. Insert the following code into the "#calculate the return for each section" area above.

In [82]:
df['return_since_last'] == df['aum_before_cash_flow'] = 0 ? 1.0 : (df['aum'] / (df['aum_before_cash_flow'] + df['cash_flow']))

SyntaxError: invalid syntax (<ipython-input-82-72aade25327e>, line 1)

Note that this does not solve every problem; the return would still be vulnerable to strange numbers if one were to sell to a very small amount and then receieve late dividends, or a mysterious increase could still get past if a positive cash flow were introduced to make it appear as if the increase was just from the cash. You could choose to handle these edge cases as well, if you'd like.

Another strange aspect of time weighted return is that, since it reflects the performance of the assets in a portfolio and not the portfolio itself, you can find that the DTWR seems extremely far off from how the client's money actually did. Take the following dataset:

In [191]:
# initialize data set. aum = assets under management
columns = ['as_of_date', 'cash_flow', 'aum']
rows = [['01-03-18', 5, 5], 
        ['01-04-18', 0, 15], 
        ['01-05-18', 10, 60], 
        ['01-06-18', 200, 262],
        ['01-07-18', 0, 206],
        ['01-08-18', -30, 168]]
df = pd.DataFrame(rows, columns=columns)

After copying the above data into the earlier code snippet: observe that while the portfolio actually lost money - by the end, 185 had been invested, while the total assets at the end were worth 168 - the daily time weighted rate of return is actually pretty good, since it doesn't reflect the investor's poor timing.

Lastly, some of our old code is written in the following format, which rearranges the previous equation.
$$
(V_n/V_0)\Pi_{1}^{n-1}(X_i) - 1
$$

where, in this case given $C_i > 0$, 
$$
X_i = 1 / (1 + C_i / V_{i-1})
$$
and if $C_i < 0$, 
$$
X_i = 1 - (C_i/V_{i})
$$

Note that if $c_i = 0$, the return doesn't need to be calculated. Translated to code (and accounting for all possible divide by 0 errors):

In [207]:
def isZero(num):
    return (num < 0.0000001 and num > -0.0000001)

def isEqualDouble(a, b):
    return isZero(a - b)

def calcreturn(prev, cf, V):
    if(isZero(cf)):
        return 1
    elif(cf > 0):
        return (1/cf) if isZero(prev) else (1.0 if isEqualDouble(cf/prev, -1.0) else (1/(1+(cf/prev))))
    else:
        return (-1 * cf) if isZero(V) else 1 - (cf/V)

df['X_i'] = np.vectorize(calcreturn)(df['aum_before_cash_flow'], df['cash_flow'], df['aum'])

v0 = 0.0
if isZero(v0):
    v0 = 1.0
vn = df.iloc[-1]['aum']
if isZero(vn):
    vn = 1.0

factor = vn / v0
print(df[['as_of_date', 'cash_flow', 'aum', 'aum_before_cash_flow', 'X_i']])
linked = factor*functools.reduce(lambda a, b: a*b, df['X_i']) - 1

print("Vn/V0 = " + str(factor))
print("DTWR: " + str(linked))

  as_of_date  cash_flow  aum  aum_before_cash_flow       X_i
0 2018-03-01          5    5                   0.0  0.200000
1 2018-04-01          0   15                   5.0  1.000000
2 2018-05-01         10   60                  15.0  0.600000
3 2018-06-01        200  262                  60.0  0.230769
4 2018-07-01          0  206                 262.0  1.000000
5 2018-08-01        -30  168                 206.0  1.178571
Vn/V0 = 168.0
DTWR: 4.483076923076923
